# 💎 다이아몬드 가격 예측 (회귀 과제)

- 빅데이터 분석기사 실기 연습을 위해 제가 만든 자료입니다.
- seaborn 라이브러리에서 load_dataset을 통해 'diamonds' 데이터셋을 가져온 후, 그것을 x_train, x_test, y_train으로 직접 분리하여 만든 데이터셋입니다. 시험 제출 양식과 유사하게 만들기 위해 각 데이터의 고유한 인덱스인 'item_id'을 추가하였습니다. 원본 데이터셋은 load_dataset('diamonds')로 불러올 수 있습니다.

### 💎 문제
- 주어진 학습용 데이터 x_train.csv를 활용하여 가격(price)을 예측하는 모형을 만들고, 평가용 데이터 x_test.csv에 적용하여 가격을 예측하여 csv 파일로 생성하시오.(평가 지표는 rmse)

### 💎 힌트
- 평가 지표로 보아 회귀 과제임을 알 수 있습니다.
- 회귀 과제에서는 predict를 사용합니다.

# 1. 패키지 불러오기

In [32]:
import pandas as pd
import numpy as np

In [33]:
import seaborn as sns

df = sns.load_dataset('diamonds')

x=df.drop(columns = 'price')
y=df['price']

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=2023)

x_test = pd.DataFrame(x_test.reset_index())
x_train = pd.DataFrame(x_train.reset_index())
y_train = pd.DataFrame(y_train.reset_index())

x_test.rename(columns = {'index':'item_id'}, inplace = True)
x_train.rename(columns = {'index':'item_id'}, inplace = True)
y_train.columns = ['item_id','price']

In [34]:
print(x_test.shape)
print(x_train.shape)
print(y_train.shape)

(10788, 10)
(43152, 10)
(43152, 2)


In [35]:
x_test.to_csv('Diamond_x_test.csv', index = False)
x_train.to_csv('Diamond_x_train.csv', index = False)
y_train.to_csv('Diamond_y_train.csv', index = False)

# 2. 데이터 읽어오기

In [73]:
x_test = pd.read_csv('./Diamond_x_test.csv')
x_train = pd.read_csv('./Diamond_x_train.csv')
y_train = pd.read_csv('./Diamond_y_train.csv')

In [74]:
x_test.head(1)

,item_id,carat,cut,color,clarity,depth,table,x,y,z
0,25388,2.02,Fair,H,SI2,56.5,61.0,8.33,8.37,4.72


In [75]:
x_train.head(1)

,item_id,carat,cut,color,clarity,depth,table,x,y,z
0,48240,0.86,Very Good,J,SI2,63.2,61.0,6.05,5.97,3.79


In [76]:
y_train.head(1)

,item_id,price
0,48240,1950


# 3. info() 함수로 기본 정보 확인하기

In [77]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10788 entries, 0 to 10787
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   item_id  10788 non-null  int64  
 1   carat    10788 non-null  float64
 2   cut      10788 non-null  object 
 3   color    10788 non-null  object 
 4   clarity  10788 non-null  object 
 5   depth    10788 non-null  float64
 6   table    10788 non-null  float64
 7   x        10788 non-null  float64
 8   y        10788 non-null  float64
 9   z        10788 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 842.9+ KB


In [78]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43152 entries, 0 to 43151
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   item_id  43152 non-null  int64  
 1   carat    43152 non-null  float64
 2   cut      43152 non-null  object 
 3   color    43152 non-null  object 
 4   clarity  43152 non-null  object 
 5   depth    43152 non-null  float64
 6   table    43152 non-null  float64
 7   x        43152 non-null  float64
 8   y        43152 non-null  float64
 9   z        43152 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 3.3+ MB


In [79]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43152 entries, 0 to 43151
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   item_id  43152 non-null  int64
 1   price    43152 non-null  int64
dtypes: int64(2)
memory usage: 674.4 KB


- 시험에서의 파일 제출 양식을 따르기 위해 x_test에 추가했던 'item_id'을 따로 분리하여 시리즈로 저장해둡니다.

In [80]:
item_id = x_test['item_id']

x_train = x_train.drop(columns='item_id')
x_test = x_test.drop(columns='item_id')
y_train = y_train.drop(columns='item_id')

# 4. 결측치 처리
- 이 데이터셋에는 결측치는 없으므로 이 단계는 통과합니다.

# 5. describe() 함수로 요약 통계량 확인
- 다른 코드 파일의 데이터셋과는 달리, 앞서 info()를 통해 이 데이터셋에는 object라는 데이터 유형이 있다는 것을 보았습니다.
- describe()는 수치형 변수에 대하여 요약 통계량을 제시해주지만, object 유형의 컬럼은 제시하지 않습니다.
- 💦 object 유형의 컬럼은 describe(include = 'object')로 변수를 지정해주어야 요약 통계량을 알 수 있습니다.

In [81]:
x_train.describe()

,carat,depth,table,x,y,z
count,43152.000000,43152.000000,43152.000000,43152.000000,43152.000000,43152.000000
mean,0.798534,61.754795,57.452181,5.731961,5.735654,3.539142
std,0.474879,1.429708,2.223297,1.122093,1.149875,0.695689
min,0.200000,43.000000,43.000000,0.000000,0.000000,0.000000
25%,0.400000,61.100000,56.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,10.740000,58.900000,8.060000


- 💦 일단 y 컬럼의 max값이 평균에 비해 다소 큰 것으로 보아 이상치로 판단됩니다. nlargest() 함수를 사용하여 이러한 이상치가 얼마나 있는지 확인합니다.

In [93]:
x_train['y'].nlargest(5)

6121     31.80
15082    10.54
6533     10.16
42041    10.10
230       9.94
Name: y, dtype: float64

- 💦 nlargest(5)의 결과, 가장 큰 값인 31.80만 이상치로 간주하고 평균값으로 바꿉니다. 

In [94]:
x_train['y'] = np.where(x_train['y']>31, x_train['y'].mean(), x_train['y'])
x_train.describe()

,carat,depth,table,x,y,z,cut_Fair,cut_Good,cut_Ideal,cut_Premium,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
count,30206.000000,30206.000000,30206.000000,30206.000000,30206.000000,30206.000000,30206.000000,30206.000000,30206.000000,30206.000000,...,30206.000000,30206.000000,30206.000000,30206.000000,30206.000000,30206.000000,30206.000000,30206.000000,30206.000000,30206.000000
mean,0.800190,61.759078,57.460541,5.735085,5.736757,3.541376,0.030954,0.091008,0.397438,0.256240,...,0.100444,0.051811,0.014004,0.031318,0.240946,0.171754,0.152652,0.228994,0.066940,0.093392
std,0.475669,1.439045,2.230419,1.123624,1.115825,0.696442,0.173196,0.287626,0.489376,0.436563,...,0.300596,0.221649,0.117508,0.174179,0.427664,0.377173,0.359657,0.420192,0.249923,0.290986
min,0.200000,43.000000,43.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.400000,61.100000,56.000000,4.710000,4.720000,2.910000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.700000,61.800000,57.000000,5.700000,5.720000,3.530000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.040000,62.500000,59.000000,6.540000,6.540000,4.040000,0.000000,0.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5.010000,79.000000,95.000000,10.740000,10.540000,6.980000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


- 그리고 object 유형의 컬럼은 따로 확인합니다.

In [82]:
x_train.describe(include = 'object')

,cut,color,clarity
count,43152,43152,43152
unique,5,7,8
top,Ideal,G,SI1
freq,17275,9067,10389


In [83]:
x_test.describe()

,carat,depth,table,x,y,z
count,10788.000000,10788.000000,10788.000000,10788.000000,10788.000000,10788.000000
mean,0.795564,61.727846,57.477197,5.727941,5.730014,3.537101
std,0.470540,1.444081,2.278709,1.120475,1.110676,0.744426
min,0.200000,50.800000,44.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,4.720000,4.730000,2.910000
50%,0.700000,61.800000,57.000000,5.690000,5.700000,3.520000
75%,1.040000,62.500000,59.000000,6.540000,6.540000,4.032500
max,4.010000,78.200000,79.000000,10.020000,9.940000,31.800000


In [84]:
x_test.describe(include = 'object')

,cut,color,clarity
count,10788,10788,10788
unique,5,7,8
top,Ideal,G,SI1
freq,4276,2225,2676


- 💦 데이터 유형이 object인 데이터의 경우에는 원-핫 인코딩을 해야 합니다. 주로 문자열이며 범주형 데이터에 속하는 이러한 데이터는 범주 간에 순서가 존재하지 않으므로 크기 비교가 의미가 없을 수 있습니다. 머신러닝으로 예측 모델을 만들기 위해서는 이러한 데이터를 수치화하여 또 하나의 변수로 사용하는 것이 좋습니다.
- 원-핫 인코딩 방법으로는 먼저 sklearn.preprocessing에서 OneHotEncoder를 사용하여 변환해주는 방법이 있습니다.
- 두번째 원-핫 인코딩 방법으로는 pandas의 get_dummies() 함수를 사용하여 변환해주는 방법이 있습니다.
- 여기서는 pandas의 get_dummies() 함수를 사용하여 범주형 변수를 수치로 바꿉니다.

In [85]:
x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)

- 💦 원-핫 인코딩 이후의 결과를 info() 함수를 통해 확인합니다. 데이터 유형이 8비트 정수(unsigned integer)로 바뀐 것을 볼 수 있습니다.

In [86]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43152 entries, 0 to 43151
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   carat          43152 non-null  float64
 1   depth          43152 non-null  float64
 2   table          43152 non-null  float64
 3   x              43152 non-null  float64
 4   y              43152 non-null  float64
 5   z              43152 non-null  float64
 6   cut_Fair       43152 non-null  uint8  
 7   cut_Good       43152 non-null  uint8  
 8   cut_Ideal      43152 non-null  uint8  
 9   cut_Premium    43152 non-null  uint8  
 10  cut_Very Good  43152 non-null  uint8  
 11  color_D        43152 non-null  uint8  
 12  color_E        43152 non-null  uint8  
 13  color_F        43152 non-null  uint8  
 14  color_G        43152 non-null  uint8  
 15  color_H        43152 non-null  uint8  
 16  color_I        43152 non-null  uint8  
 17  color_J        43152 non-null  uint8  
 18  clarit

# 6. 학습용 / 검증용 데이터셋 분리
- 회귀 과제에서는 stratify를 사용하지 않습니다. 이는 클래스의 비율이 중요한 분류 과제에서 클래스의 비율을 유지하기 위해 사용하는 것으로서, 연속형 값을 예측하는 회귀 과제에는 필요하지 않습니다.

In [87]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train['price'], 
                                                      test_size = 0.3)

# 7. 모델 학습
- 랜덤 포레스트 모델을 사용합니다. 회귀 과제에서는 RandomForestRegressor()를 사용합니다.

In [88]:
from sklearn.ensemble import RandomForestRegressor

modelRF = RandomForestRegressor()
modelRF.fit(x_train, y_train)

RandomForestRegressor()

# 8. 모델 평가
- 평가 지표로 rmse를 사용합니다.

In [89]:
y_validation_predict = modelRF.predict(x_valid)

In [90]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_valid, y_validation_predict)
rmse = mean_squared_error(y_valid, y_validation_predict, squared = False)
r2score = r2_score(y_valid, y_validation_predict)

print(mse)
print(rmse)
print(r2score)

319559.52353511384
565.2959610107911
0.9797495860509339


In [91]:
pred = modelRF.predict(x_test)

# 9. 파일 제출

In [92]:
result =  pd.DataFrame({'item_id':item_id, 'result':pred}).to_csv('diamond_sub.csv', index=False)
df2 =  pd.read_csv('./diamond_sub.csv')
df2

,item_id,result
0,25388,14156.53
1,15396,5086.54
2,35778,892.83
3,3379,446.84
4,45160,1508.62
...,...,...
10783,36438,939.28
10784,50067,2172.64
10785,29605,722.92
10786,19050,8750.66
